In [94]:
import requests
from bs4 import BeautifulSoup
import json
import matplotlib.pyplot as plt

def get_description(url):
    response = requests.get(url)
    if response.status_code == 200:
        # Parse the HTML content
        soup = BeautifulSoup(response.content, "html.parser")
        # Find all image tags
        images = soup.find_all("img")
        script = soup.find_all("script", type="application/ld+json")[0]
        description = json.loads(script.string)['description'] 
    else:
        print(f"Failed to fetch the page. HTTP Status Code: {response.status_code}")
    return description

In [ ]:
import numpy as np
from io import BytesIO
from PIL import Image
def image_url_to_numpy_array(url, new_width=1080, new_height=1350):
    response = requests.get(url)
    response.raise_for_status()  # Ensure the request was successful
    im = Image.open(BytesIO(response.content))
    width, height = im.size
    aspect_ratio_original = width / height
    aspect_ratio_new = new_width / new_height
    if aspect_ratio_original > aspect_ratio_new:
        scale_factor = new_height / height
    else:
        scale_factor = new_width / width
    target_width = int(width * scale_factor)
    target_height = int(height * scale_factor)
    im = im.resize((target_width, target_height), Image.Resampling.LANCZOS)
    left = (target_width - new_width) / 2
    top = (target_height - new_height) / 2
    right = left + new_width
    bottom = top + new_height
    im = im.crop((left, top, right, bottom))
    img_array = np.array(im)
    return img_array


In [90]:
import requests
import xml.etree.ElementTree as ET
import re

# URL of the RSS feed
url = "https://www.indiatoday.in/rss/1206614"

# Fetch the RSS feed
response = requests.get(url)

rss = {}

if response.status_code == 200:
    # Parse the XML content
    root = ET.fromstring(response.content)
    
    # Iterate through the items in the feed
    for idx, item in enumerate(root.findall(".//item")):
        title = item.find("title").text
        link = item.find("link").text[:-1]
        description = item.find("description").text

        rss[idx] = {"Title": title, "Link": link, "Image": re.search("(?P<url>https?://[^\s]+)", description.split("src")[-1]).group("url")[:-2], "Description": get_description(link)}
        
        # print(f"Title: {title}")
        # print(f"Link: {link}")
        # print(f"Image: {re.search("(?P<url>https?://[^\s]+)", description.split("src")[-1]).group("url")}")
else:
    print(f"Failed to fetch the RSS feed. HTTP Status Code: {response.status_code}")

In [137]:
rss_choosen = rss[8]
rss_choosen

{'Title': 'Madhabi Puri Buch | Trial by fire',
 'Link': ' https://www.indiatoday.in/magazine/the-big-story/story/20240923-madhabi-puri-buch-trial-by-fire-2599237-2024-09-14?utm_source=rss',
 'Image': 'https://akm-img-a-in.tosshub.com/indiatoday/images/story/202409/trial-by-fire-135611272-16x9_0.jpg?VersionId=G3_9375i2wnMumOKwBp8VkJ_i8F7X5lm',
 'Description': 'Hounded by allegations of conflict of interest in the Adani probe and rising discontent among her own staff, the Sebi chief has much to answer for—and disclose'}

In [138]:
from PIL import Image, ImageDraw, ImageFont
import textwrap

fill_image = image_url_to_numpy_array(rss_choosen["Image"])

template = plt.imread("template.png")
overlay = Image.open("template.png").convert("RGBA")
background = Image.fromarray(fill_image)
background.paste(overlay, (0,0), overlay)

image = background

draw = ImageDraw.Draw(image)

font = ImageFont.truetype("Nunito-VariableFont_wght.ttf", 50)
text = rss_choosen["Title"]
image_width, image_height = image.size
text_width, text_height = draw.textsize(text, font=font)
position = ((image_width - text_width) // 2, 1000)
draw.text(position, text, font=font, fill="white")  # Adjust `fill` as needed

text = rss_choosen["Description"]
font = ImageFont.truetype("Nunito-VariableFont_wght.ttf", 30)

# Define the maximum width for the text
max_width = image_width - 0  # Leave some padding on both sides

# Use textwrap to split the text into multiple lines
lines = textwrap.wrap(text, width=70)  # Adjust `width` based on text and font size

# Calculate line height
line_height = draw.textsize("A", font=font)[1]

# Start drawing lines
y = 1100  # Starting y position
for line in lines:
    text_width, _ = draw.textsize(line, font=font)
    x = (image_width - text_width) // 2  # Center align each line
    draw.text((x, y), line, font=font, fill="white")
    y += line_height + 5 

image.save("result.png")

/var/folders/3p/xlr6tgyx4t980qpxnnrs12kc0000gn/T/ipykernel_99902/1287201412.py:18: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  text_width, text_height = draw.textsize(text, font=font)
/var/folders/3p/xlr6tgyx4t980qpxnnrs12kc0000gn/T/ipykernel_99902/1287201412.py:32: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  line_height = draw.textsize("A", font=font)[1]
/var/folders/3p/xlr6tgyx4t980qpxnnrs12kc0000gn/T/ipykernel_99902/1287201412.py:37: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  text_width, _ = draw.textsize(line, font=font)
/var/folders/3p/xlr6tgyx4t980qpxnnrs12kc0000gn/T/ipykernel_99902/1287201412.py:37: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  text_